In [1]:
from openai import OpenAI
from tqdm import tqdm
import time
import json
import os

from dotenv import load_dotenv
load_dotenv()

with open('../datas/post_preprocessed_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"총 {len(data)}건")
print(json.dumps(data[0], ensure_ascii=False, indent=2))            

총 838건
{
  "ids": "groro_3300",
  "question": "핑크프린세스 잎이 왜 이렇게 되었을까요? 햇볕도 잘 보여주고 통풍도 잘 시키고 있는데 며칠전부터 이렇네요 ㅜㅜ",
  "answer": "[COMMENT1] 저는 다른 잎 또한 번진 것으로 보아 병충해의 가능성도 볼 수 있어서 약을 한 번 치시고 더 번져나가는지 지켜보시는 것도 좋으실 것 같아요 다른 잎들도 모두 그런 것이 아니라 한 두 잎에서 번져나가는 것처럼 보여서요!\n[COMMENT2] 다른 잎도 증세를 보이는 것으로 보아 물과 연관된 상황이라고 보여집니다공중 습도가 어느 정도 있어야 하고, 해를 보여주는 시간을 아주 조금만 줄여보시는 것도 좋아보여요\n[COMMENT3] 쟤가 유독 잘저래요! 통풍 잘되면 좀 나을거예요!\n[COMMENT4] 혹시 다른 잎도 그렇다면 전체적인 수분 부족일 경우가 큽니다! 햇빛과 바람 모두 잘 통하니까 그만큼 수분이 금방 부족해지겠죠,,근데 유독 한 잎만 그렇고 그게 햇빛 방향이라면 빛에 의한 장해일 수 있으니 직접적으로 빛이 닿지 않는 곳으로 옮겨주세요!\n[COMMENT5] 여러가지 방법을 시도해보세요!! 다른 분들이 말씀하신 것처럼 여러 이유가 있으니까요!\n[COMMENT6] 잎변색은 정말 여러가지 원인이 있어요 과습이거나 반대로 물부족도 있고, 영양부족이나 병충해도 있어요 잎에 생긴 수포가 햇볕에 탔거나, 직광에 힘들어서 그럴수도 있고요 필로덴드론과는 뿌리는 조금 건조하게 공중습도는 건조하지 않게 키워주시고, 빛은 간접광을 좋아해 빛 잘드는 창가 안쪽으로 조금 들어오는 공간이 좋습니다\n[COMMENT7] 고맙습니다\n[COMMENT8] 햇볕과 통풍 잘 시켜주시는것은 매우 잘하시는것 같아요! 그런데 혹시 물은 얼마나 주시나요? 보통은 수분이 부족할때 잎이 저렇게 누렇게 변하더라구요 혹시 흙이 너무 말랐다싶으면 목말라서 그런걸수도있으니까요 천천히 수분 공급해주시고 지켜봐주세요!\n[COMMENT9] 네 감사합니다",
  "metad

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 시스템 프롬프트 정의
SYSTEM_PROMPT = """
##당신의 역할:
- 당신은 "식물 상담 Q&A 데이터"를 RAG에 맞게 정리하는 데이터 큐레이터입니다.
- 각 레코드(질문/답변/기본 메타데이터)를 받아,
  1) 모호하거나 정보가 부족한 질문을 식별하고,
  2) multi-label 카테고리 태깅을 수행한 뒤,
  3) 최소한의 RAG 메타데이터를 생성합니다.

##반드시 지켜야 할 원칙:
- 응답은 **반드시 하나의 JSON 객체**로만 반환합니다.
- JSON 이외의 설명, 코멘트, 자연어 문장은 절대 넣지 마세요.
- Boolean은 `true`/`false` 로 표기합니다.
- 알 수 없는 값은 `null` 또는 빈 배열 `[]` 을 사용합니다.

[1단계: 모호/저정보(low-context) 질문 판별 및 제외]
다음 기준을 종합적으로 고려해 질문이 low-context / image-dependent 인지 판단합니다.

###**low-context / image-dependent** 로 판단하는 경우:
- 질문이 너무 짧고 구체 정보가 거의 없음
  - 예: "무슨 식물일까요?", "이거 왜 이럴까요?", "궁금합니다"
- 텍스트만 보고는 상태/환경을 유추하기 어려움
- 질문의 핵심이 사진에만 의존하는 경우

###필드 정의:
- `is_low_context`: 위 기준에 해당하면 true, 아니면 false
- `is_image_dependent`: 사진 없이는 의미 파악이 어렵다면 true, 아니면 false
- `is_off_domain`: 식물 상담과 무관한 질문이면 true, 아니면 false (식물 관련이면 false)

###`exclude` 규칙:
- 다음 중 하나라도 참이면 `exclude = true` 로 설정합니다.
  - `is_low_context == true`
  - `is_off_domain == true`
- 그렇지 않으면 `exclude = false` 로 설정합니다.

※ `exclude = true` 인 레코드는 RAG 검색용으로 쓰지 않을 예정이므로 카테고리는 null/빈 배열로 둡니다.

[2단계: Q&A 카테고리 multi-label 분류]
하나의 질문이 여러 카테고리에 동시에 속할 수 있습니다.

- `category_main`: 가장 대표적인 카테고리 1개 (string)
- `category_sub`: 나머지 관련 카테고리 리스트 (string 배열, 없으면 빈 배열)

###카테고리 정의:
1) "symptom_diagnosis" (증상 진단 / 문제 원인 진단)
2) "environment_recommendation" (환경/관리 방법 추천)
3) "urgent_care" (시급한 조치)
4) "plant_identification" (식물/품종/이름 확인)
5) "repotting" (분갈이/화분/흙)
6) "propagation" (번식/삽목/수경)
7) "blooming_fruiting" (개화/열매)
8) "pruning_shaping" (가지치기/수형 관리)
9) "beginner_faq" (초보자/입문자 관점 질문)
10) "other" (위에 명확히 들어맞지 않는 경우)

###규칙:
- `exclude == true` 인 경우:
  - `category_main` 은 null
  - `category_sub` 는 빈 배열 `[]`
- `exclude == false` 인 경우:
  - 질문 내용에 가장 잘 맞는 카테고리를 `category_main` 으로 1개 선택
  - 추가로 관련 있는 카테고리를 `category_sub` 배열에 넣습니다.
  - 어느 것도 맞지 않을 경우 `category_main` 을 "other" 로 설정하고 `category_sub` 는 빈 배열로 둡니다.

[3단계: 최종 JSON 출력]
입력 레코드를 기반으로, 아래 스키마를 만족하는 JSON 객체 **한 개**만 출력하세요.

{
  "ids": string,
  "question": string,
  "answer": string,
  "base_metadata": {
    "post_id": string | null,
    "channel": string | null,
    "hashtags": string | null,
    "date": string | null,
    "source_platform": string | null
  },
  "exclude": boolean,
  "category_main": string | null,
  "category_sub": [string],
  "is_low_context": boolean,
  "is_image_dependent": boolean,
  "is_off_domain": boolean
}

###다시 한 번 강조합니다:
- JSON 이외의 어떤 텍스트도 출력하지 마세요.
- 위 스키마를 벗어나지 마세요.
"""

In [3]:
def process_record(record, model="gpt-4o-mini", max_retries=3):

    # 입력 레코드 구성
    input_data = {
        "ids": record["ids"],
        "question": record["question"],
        "answer": record["answer"],
        "metadata": record["metadata"]
    }
    
    user_message = json.dumps(input_data, ensure_ascii=False, indent=2)
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_message}
                ],
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            # 결과 검증
            required_fields = ["ids", "question", "answer", "base_metadata", 
                             "exclude", "category_main", "category_sub",
                             "is_low_context", "is_image_dependent", "is_off_domain"]
            
            if all(field in result for field in required_fields):
                return result
            else:
                print(f"[WARN] 누락된 필드가 있습니다: {record['ids']}")
                if attempt < max_retries - 1:
                    time.sleep(1)
                    continue
                else:
                    return None
                    
        except json.JSONDecodeError as e:
            print(f"[ERROR] JSON 파싱 실패 ({record['ids']}): {e}")
            if attempt < max_retries - 1:
                time.sleep(1)
                continue
            else:
                return None
                
        except Exception as e:
            print(f"[ERROR] 처리 중 문제 발생 ({record['ids']}): {e}")
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            else:
                return None
    
    return None

In [4]:
# 전체 배치 처리 => openai api rate limit
def process_batch(data, start_idx=0, end_idx=None, output_file='./post_preprocessed_data_v2.json', save_interval=50, model="gpt-4o-mini"):
    """
    데이터를 배치로 처리하고 주기적으로 저장
    
    Args:
        data: 처리할 데이터 리스트
        start_idx: 시작 인덱스
        end_idx: 종료 인덱스 (None이면 끝까지)
        output_file: 결과 저장 파일
        save_interval: 저장 주기
        model: 사용할 모델
    """
    if end_idx is None:
        end_idx = len(data)
    
    results = []
    failed_records = []
    
    # 기존 결과 파일이 있으면 로드
    if os.path.exists(output_file):
        try:
            with open(output_file, 'r', encoding='utf-8') as f:
                results = json.load(f)
            print(f"기존 결과 {len(results)}개를 로드했습니다.")
        except:
            print("기존 결과 파일을 로드할 수 없습니다. 새로 시작합니다.")
            results = []
    
    print(f"\n처리 범위: {start_idx} ~ {end_idx} (총 {end_idx - start_idx}개)")
    print(f"모델: {model}")
    print(f"결과 저장 파일: {output_file}")
    print(f"저장 주기: {save_interval}개마다\n")
    
    for idx in tqdm(range(start_idx, end_idx), desc="Processing"):
        record = data[idx]
        
        result = process_record(record, model=model)
        
        if result:
            results.append(result)
        else:
            failed_records.append({
                "idx": idx,
                "ids": record["ids"],
                "question": record["question"][:100]
            })
            print(f"\n실패: {idx} - {record['ids']}")
        
        # 주기적으로 저장
        if (idx - start_idx + 1) % save_interval == 0:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\n중간 저장 완료: {len(results)}개")
        
        # API Rate Limit 방지
        time.sleep(0.5)
    
    # 최종 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # 실패 레코드 저장
    if failed_records:
        failed_file = output_file.replace('.json', '_failed.json')
        with open(failed_file, 'w', encoding='utf-8') as f:
            json.dump(failed_records, f, ensure_ascii=False, indent=2)
        print(f"\n실패한 레코드 {len(failed_records)}개를 {failed_file}에 저장했습니다.")
    
    print(f"\n처리 완료!")
    print(f"  - 성공: {len(results)}개")
    print(f"  - 실패: {len(failed_records)}개")
    print(f"  - 결과 파일: {output_file}")
    
    return results, failed_records


- 실행

In [69]:
results, failed = process_batch(data, start_idx= 831, end_idx=838, 
                                 output_file='./post_preprocessed_data_v2.json',
                                 save_interval=5)

기존 결과 829개를 로드했습니다.

처리 범위: 831 ~ 838 (총 7개)
모델: gpt-4o-mini
결과 저장 파일: ./post_preprocessed_data_v2.json
저장 주기: 5개마다



Processing:  57%|█████▋    | 4/7 [00:44<00:33, 11.33s/it]


중간 저장 완료: 834개


Processing: 100%|██████████| 7/7 [01:13<00:00, 10.50s/it]


처리 완료!
  - 성공: 836개
  - 실패: 0개
  - 결과 파일: ./post_preprocessed_data_v2.json


In [70]:
len(results), len(failed)

(836, 0)

In [22]:
# 결과 분석 함수
def analyze_results(result_file):

    with open(result_file, 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    total = len(results)
    excluded = sum(1 for r in results if r['exclude'])
    included = total - excluded
    
    print(f"=" * 80)
    print(f"결과 분석")
    print(f"=" * 80)
    print(f"\n전체 통계")
    print(f"  - 전체 레코드: {total}개")
    print(f"  - RAG 사용 가능: {included}개 ({included/total*100:.1f}%)")
    print(f"  - RAG 제외: {excluded}개 ({excluded/total*100:.1f}%)")
    
    # exclude 이유 분석
    low_context = sum(1 for r in results if r['is_low_context'])
    image_dependent = sum(1 for r in results if r['is_image_dependent'])
    off_domain = sum(1 for r in results if r['is_off_domain'])
    
    print(f"\n제외 이유별 통계 (중복 가능)")
    print(f"  - Low Context: {low_context}개 ({low_context/total*100:.1f}%)")
    print(f"  - Image Dependent: {image_dependent}개 ({image_dependent/total*100:.1f}%)")
    print(f"  - Off Domain: {off_domain}개 ({off_domain/total*100:.1f}%)")
    
    # 카테고리별 통계
    category_main_count = {}
    category_sub_count = {}
    
    for r in results:
        if not r['exclude'] and r['category_main']:
            category_main_count[r['category_main']] = category_main_count.get(r['category_main'], 0) + 1
            
            for sub_cat in r['category_sub']:
                category_sub_count[sub_cat] = category_sub_count.get(sub_cat, 0) + 1
    
    print(f"\n메인 카테고리별 통계")
    for cat, count in sorted(category_main_count.items(), key=lambda x: -x[1]):
        print(f"  - {cat}: {count}개 ({count/included*100:.1f}%)")
    
    print(f"\n서브 카테고리별 통계 (중복 포함)")
    for cat, count in sorted(category_sub_count.items(), key=lambda x: -x[1]):
        print(f"  - {cat}: {count}개")
    
    return {
        'total': total,
        'included': included,
        'excluded': excluded,
        'low_context': low_context,
        'image_dependent': image_dependent,
        'off_domain': off_domain,
        'category_main_count': category_main_count,
        'category_sub_count': category_sub_count
    }


In [73]:
# 결과 분석 실행
stats = analyze_results('./post_preprocessed_data_v2.json')

결과 분석

전체 통계
  - 전체 레코드: 836개
  - RAG 사용 가능: 823개 (98.4%)
  - RAG 제외: 13개 (1.6%)

제외 이유별 통계 (중복 가능)
  - Low Context: 9개 (1.1%)
  - Image Dependent: 22개 (2.6%)
  - Off Domain: 2개 (0.2%)

메인 카테고리별 통계
  - symptom_diagnosis: 416개 (50.5%)
  - environment_recommendation: 96개 (11.7%)
  - plant_identification: 94개 (11.4%)
  - repotting: 55개 (6.7%)
  - urgent_care: 52개 (6.3%)
  - propagation: 45개 (5.5%)
  - blooming_fruiting: 35개 (4.3%)
  - pruning_shaping: 23개 (2.8%)
  - beginner_faq: 4개 (0.5%)
  - other: 3개 (0.4%)

서브 카테고리별 통계 (중복 포함)
  - environment_recommendation: 548개
  - urgent_care: 311개
  - symptom_diagnosis: 146개
  - other: 82개
  - repotting: 63개
  - beginner_faq: 53개
  - pruning_shaping: 32개
  - propagation: 30개
  - blooming_fruiting: 28개
  - plant_identification: 18개
  - watering: 1개


In [24]:
# RAG에 사용 가능한 데이터만 추출 (exclude=false)
def get_rag_ready_data(result_file, output_file=None):

    with open(result_file, 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    rag_ready = [r for r in results if not r['exclude']]
    
    print(f"RAG 사용 가능 데이터: {len(rag_ready)}개")
    
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(rag_ready, f, ensure_ascii=False, indent=2)
        print(f"결과를 {output_file}에 저장했습니다.")
    
    return rag_ready

- 최종 rag 데이터셋

In [74]:
get_rag_ready_data('./post_preprocessed_data_v2.json', './post_preprocessed_data_v3.json')

RAG 사용 가능 데이터: 823개
결과를 ./post_preprocessed_data_v3.json에 저장했습니다.


[{'ids': 'groro_3300',
  'question': '핑크프린세스 잎이 왜 이렇게 되었을까요? 햇볕도 잘 보여주고 통풍도 잘 시키고 있는데 며칠전부터 이렇네요 ㅜㅜ',
  'answer': '[COMMENT1] 저는 다른 잎 또한 번진 것으로 보아 병충해의 가능성도 볼 수 있어서 약을 한 번 치시고 더 번져나가는지 지켜보시는 것도 좋으실 것 같아요 다른 잎들도 모두 그런 것이 아니라 한 두 잎에서 번져나가는 것처럼 보여서요!\n[COMMENT2] 다른 잎도 증세를 보이는 것으로 보아 물과 연관된 상황이라고 보여집니다공중 습도가 어느 정도 있어야 하고, 해를 보여주는 시간을 아주 조금만 줄여보시는 것도 좋아보여요\n[COMMENT3] 쟤가 유독 잘저래요! 통풍 잘되면 좀 나을거예요!\n[COMMENT4] 혹시 다른 잎도 그렇다면 전체적인 수분 부족일 경우가 큽니다! 햇빛과 바람 모두 잘 통하니까 그만큼 수분이 금방 부족해지겠죠,,근데 유독 한 잎만 그렇고 그게 햇빛 방향이라면 빛에 의한 장해일 수 있으니 직접적으로 빛이 닿지 않는 곳으로 옮겨주세요!\n[COMMENT5] 여러가지 방법을 시도해보세요!! 다른 분들이 말씀하신 것처럼 여러 이유가 있으니까요!\n[COMMENT6] 잎변색은 정말 여러가지 원인이 있어요 과습이거나 반대로 물부족도 있고, 영양부족이나 병충해도 있어요 잎에 생긴 수포가 햇볕에 탔거나, 직광에 힘들어서 그럴수도 있고요 필로덴드론과는 뿌리는 조금 건조하게 공중습도는 건조하지 않게 키워주시고, 빛은 간접광을 좋아해 빛 잘드는 창가 안쪽으로 조금 들어오는 공간이 좋습니다\n[COMMENT7] 고맙습니다\n[COMMENT8] 햇볕과 통풍 잘 시켜주시는것은 매우 잘하시는것 같아요! 그런데 혹시 물은 얼마나 주시나요? 보통은 수분이 부족할때 잎이 저렇게 누렇게 변하더라구요 혹시 흙이 너무 말랐다싶으면 목말라서 그런걸수도있으니까요 천천히 수분 공급해주시고 지켜봐주세요!\n[COMMENT9] 네 감사합니다',
  'base_metadata'

- 파인콘 적재

In [75]:
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict, Counter
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Tuple
import numpy as np
import pandas as pd
import json
import os

notebook_dir = Path.cwd()
if notebook_dir.name != 'cache':
    os.chdir(Path(__file__).parent if '__file__' in globals() else Path.cwd())

print(f"현재 작업 디렉토리: {os.getcwd()}")

# 데이터 로드
with open('post_preprocessed_data_v3.json', 'r', encoding='utf-8') as f:
    documents = json.load(f)

print(f"전체 문서 수: {len(documents)}")
print(f"제외되지 않은 문서 수: {len([d for d in documents if not d['exclude']])}")

# 카테고리 분포 확인
category_counts = Counter([d['category_main'] for d in documents if not d['exclude']])
print("\n카테고리 분포:")
for cat, count in category_counts.most_common():
    print(f"  {cat}: {count}")

현재 작업 디렉토리: c:\SKN_19\project\llm\cache
전체 문서 수: 823
제외되지 않은 문서 수: 823

카테고리 분포:
  symptom_diagnosis: 416
  environment_recommendation: 96
  plant_identification: 94
  repotting: 55
  urgent_care: 52
  propagation: 45
  blooming_fruiting: 35
  pruning_shaping: 23
  beginner_faq: 4
  other: 3


In [79]:
# Pinecone VectorDB에 데이터 적재
import sys
sys.path.append('..')

from modules.config import pc, embeddings
from pinecone import ServerlessSpec

print("=" * 80)
print("Pinecone VectorDB 데이터 적재 시작")
print("=" * 80)

# 적재할 문서만 필터링 (exclude=False)
docs_to_upsert = [d for d in documents if not d['exclude']]
print(f"\n적재 대상 문서 수: {len(docs_to_upsert)}개")

QNA_INDEX_NAME = "plant-qna-v3"
DIMENSION = 1536  
UPSERT_NAMESPACE = f"{QNA_INDEX_NAME}-openai"

if QNA_INDEX_NAME not in [idx["name"] for idx in pc.list_indexes()]:
    print(f"\n'{QNA_INDEX_NAME}' 인덱스 생성 중...")
    pc.create_index(
        name=QNA_INDEX_NAME,
        dimension=DIMENSION,
        metric='cosine',
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"'{QNA_INDEX_NAME}' 인덱스 생성 완료")
else:
    print(f"\n'{QNA_INDEX_NAME}' 인덱스 이미 존재")

index = pc.Index(QNA_INDEX_NAME)
print(f"인덱스 연결 완료")

# 기존 데이터 확인
existing_stats = index.describe_index_stats()
print(f"\n현재 인덱스 상태:")
print(f"  - 총 벡터 수: {existing_stats.get('total_vector_count', 0)}")
print(f"  - 네임스페이스: {list(existing_stats.get('namespaces', {}).keys())}")

# 사용자 확인
print(f"\n적재 설정:")
print(f"  - 네임스페이스: {UPSERT_NAMESPACE}")
print(f"  - 문서 수: {len(docs_to_upsert)}개")

response = input("\n데이터를 적재하시겠습니까? (y/n): ")

if response.lower() == 'y':
    print("\n임베딩 및 적재 시작...")
    
    # 임베딩할 텍스트 생성
    texts_to_embed = []
    for doc in docs_to_upsert:
        text = f"Question: {doc['question']}\nAnswer: {doc['answer']}"
        texts_to_embed.append(text)
    
    # 벡터 임베딩 생성
    print(f"OpenAI 임베딩 생성 중... ({len(texts_to_embed)}개)")
    vectors = embeddings.embed_documents(texts_to_embed)
    print(f"임베딩 완료")
    
    # Pinecone 적재 데이터 준비
    insert_data = []
    for doc, vector in zip(docs_to_upsert, vectors):
        metadata = {
            'ids': doc['ids'],
            'text': f"Question: {doc['question']}\nAnswer: {doc['answer']}",
            'question': doc['question'],
            'answer': doc['answer'][:500],
            'category_main': doc['category_main'],
            'category_sub': doc.get('category_sub', []),
            # base_metadata
            'post_id': doc['base_metadata']['post_id'],
            'channel': doc['base_metadata']['channel'],
            'date': doc['base_metadata']['date'],
            'source_platform': doc['base_metadata']['source_platform']
        }
        
        insert_data.append({
            'id': doc['ids'],
            'values': vector,
            'metadata': metadata
        })
    
    # 배치 적재 (한 번에 최대 100개)
    def batch(iterable, n=100):
        for idx in range(0, len(iterable), n):
            yield iterable[idx: idx + n]
    
    print(f"\nPinecone에 배치 업로드 중...")
    for idx, batch_data in enumerate(batch(insert_data, 100)):
        index.upsert(
            vectors=batch_data,
            namespace=UPSERT_NAMESPACE
        )
        print(f"  Batch {idx+1}/{(len(insert_data)-1)//100 + 1} 완료 ({len(batch_data)}개)")
    
    print(f"\n총 {len(insert_data)}개 문서 적재 완료!")
    print(f"   네임스페이스: {UPSERT_NAMESPACE}")
    
    # 적재 후 통계 확인
    updated_stats = index.describe_index_stats()
    print(f"\n업데이트된 인덱스 상태:")
    print(f"  - 총 벡터 수: {updated_stats.get('total_vector_count', 0)}")
    for ns, info in updated_stats.get('namespaces', {}).items():
        print(f"  - {ns}: {info.get('vector_count', 0)}개")
        
else:
    print("\n적재를 건너뜁니다.")
    print(f"   기존 네임스페이스를 사용하거나, 아래에서 UPSERT_NAMESPACE를 수동으로 설정하세요.")

print("\n" + "=" * 80)


Pinecone VectorDB 데이터 적재 시작

적재 대상 문서 수: 823개

'plant-qna-v3' 인덱스 이미 존재
인덱스 연결 완료

현재 인덱스 상태:
  - 총 벡터 수: 807
  - 네임스페이스: ['plant-qna-v3-openai']

적재 설정:
  - 네임스페이스: plant-qna-v3-openai
  - 문서 수: 823개

임베딩 및 적재 시작...
OpenAI 임베딩 생성 중... (823개)
임베딩 완료

Pinecone에 배치 업로드 중...
  Batch 1/9 완료 (100개)
  Batch 2/9 완료 (100개)
  Batch 3/9 완료 (100개)
  Batch 4/9 완료 (100개)
  Batch 5/9 완료 (100개)
  Batch 6/9 완료 (100개)
  Batch 7/9 완료 (100개)
  Batch 8/9 완료 (100개)
  Batch 9/9 완료 (23개)

총 823개 문서 적재 완료!
   네임스페이스: plant-qna-v3-openai

업데이트된 인덱스 상태:
  - 총 벡터 수: 807
  - plant-qna-v3-openai: 807개

